In [1]:
# Import useful libraries
import ee, ipywidgets, datetime, folium, utils, concurrent.futures, sentinel_satellites
from folium.plugins import Draw

# Manure date detector 🌱
The objective of this notebook is to create an interactive application that can allow to obtain (in a simple manner) when a crop field have been manured (it works also with multiple crops).

This notebooks gathers all the results and observations that have been understood in a previous phase.

## Select the crop field/s of interest
In the following line, through the interactive map, you can draw a polygon that represent your crop field of interest (it works also if you select multiple crops).<br>
Once all the crop fields have been selected, you just need to click on the **Export** button that you can find on the upper right part of the map. <br>
**Please add the new created JSON file inside the tests folder (within the same folder where you also have this notebook).**

In [2]:
# Create a map centered at a certain location
m = folium.Map(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}', attr='Tiles &copy; Esri')

# Add the drawing tool to the map
draw = Draw(export=True, filename='test-fields.json')
draw.add_to(m)

# Display the map
m

## Select dates
Through the following widgets you have to choose within which range of dates you want to check whether the crop fields have been manured.

In [3]:
start_date_widget = ipywidgets.widgets.DatePicker(description='Start date', value=(datetime.date.today() - datetime.timedelta(days=365)), disabled=False)
display(start_date_widget)

end_date_widget = ipywidgets.widgets.DatePicker(description='End date', value=datetime.date.today(), disabled=False)
display(end_date_widget)

DatePicker(value=datetime.date(2022, 4, 4), description='Start date')

DatePicker(value=datetime.date(2023, 4, 4), description='End date')

In [4]:
fields_df = utils.get_fields_df('./test-fields.json')
fields_df

,crop_field_name,polygon_coordinates
0,P-Test1,"[(9.882513, 45.522135), (9.88276, 45.520233), ..."
1,P-Test2,"[(-2.333125, 52.358724), (-2.332406, 52.357905..."
2,P-Test3,"[(112.132119, 31.349431), (112.132317, 31.3491..."


## Extracting features from the crop-field/s specified
Now the objective is to gather useful features that will be further used in order to understand when the selected crop fields have been manured, within the specified range of dates.<br>
In order to extract efficiently the features, [sentinel-satellites](https://pypi.org/project/sentinel-satellites/) PyPI library has been used. Read the documentation in order to choose correctly the `fields_thread` parameter value.<br>
Since the mentioned library relies on Google Earth Engine APIs you have to authenticate to GEE with your account (if you don't already have an account go at the [earth-engine website](https://earthengine.google.com/)).

In [5]:
# Initializes the Google Earth Engine APIs
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [6]:
# Select the fields thread widget
fields_thread_widget = ipywidgets.widgets.IntSlider(value=1, min=1, max=concurrent.futures.ThreadPoolExecutor()._max_workers, step=1, description='FThreads:')
fields_thread_widget

IntSlider(value=1, description='FThreads:', max=12, min=1)

In [9]:
# Sentinel-2 features extraction
s2_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=2, filters_params=['40'], fields_threads=fields_thread_widget.value).reset_index(drop=True)
s2_features_extracted_df

,crop_field_name,s2_acquisition_date,B1,B2,B3,B4,B5,B6,B7,B8,...,NDRE3,CARI1,CARI2,MCARI,MCARI1,MCARI2,BSI,GLI,ALTERATION,SDI
0,P-Test1,2022-04-08,1206.754738,1164.292502,1525.776710,1266.796759,1978.261741,4600.674540,5671.274924,5823.562757,...,0.013248,4877.443069,3360.074198,969.719239,6965.751761,0.747614,10988.114546,0.113169,1.437438,3923.157649
1,P-Test1,2022-04-11,1369.561110,1327.585279,1664.784125,1433.656962,2061.355397,4435.025542,5391.594617,5711.366932,...,0.028801,4868.477175,3777.632886,788.483381,6520.460731,0.713571,11219.568763,0.093309,1.354596,3684.728646
2,P-Test1,2022-04-13,1331.766548,1355.815161,1670.180445,1384.466630,2119.022796,4650.236748,5831.130733,6100.148860,...,0.022547,5173.590165,3727.804019,986.892566,7236.295963,0.741436,11769.514964,0.098687,1.418098,4035.865971
3,P-Test1,2022-04-16,1241.152431,1271.920901,1563.224938,1326.109585,1926.724526,4254.374073,5304.558912,5621.006042,...,0.028964,4583.587898,3284.941985,767.015663,6554.550849,0.728833,10930.938713,0.092309,1.404244,3691.003296
4,P-Test1,2022-04-18,1267.569349,1223.586927,1598.960450,1324.328207,2094.182093,4649.344686,5698.498764,6048.823400,...,0.029822,5192.296598,3683.799734,1060.762247,7231.699379,0.747252,11495.309217,0.113126,1.436652,4032.435595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,P-Test3,2023-01-27,3055.691358,3059.086420,3265.987654,3090.074074,3735.827160,5139.345679,5407.975309,5747.555556,...,0.030441,8055.082079,10681.609344,667.095362,4101.198519,0.445991,15250.243560,0.030188,1.255897,3078.419753
118,P-Test3,2023-02-01,2328.592593,2493.901235,2554.938272,2520.530864,2975.962963,3805.246914,4063.061728,4242.432099,...,0.021597,6331.268276,8257.494805,438.303810,2533.213333,0.373217,12429.924130,0.009427,1.281483,1767.271605
119,P-Test3,2023-02-06,2077.061728,2091.716049,2331.407407,2343.530864,2981.086420,4139.493827,4397.246914,4601.271605,...,0.022673,6514.507438,9626.449595,645.717270,3232.234074,0.439064,12274.298202,0.025013,1.281400,2075.456790
120,P-Test3,2023-02-26,2594.271605,2486.246914,2600.716049,2431.185185,2894.493827,4195.271605,4489.086420,4703.222222,...,0.023295,6230.095416,6398.521718,481.648382,3536.201481,0.469942,12618.139490,0.028066,1.263242,2331.419753


In [10]:
# Sentinel-1 features extraction
s1_features_extracted_df = sentinel_satellites.get_features(fields_df, start_date_widget.value, end_date_widget.value, sentinel=1, filters_params=['ASCENDING'], fields_threads=fields_thread_widget.value).reset_index(drop=True)
s1_features_extracted_df

,crop_field_name,s1_acquisition_date,VV,VH,AVE,DIF,RAT1,RAT2,NDI,RVI
0,P-Test1,2022-04-15,-17.694409,-24.523803,-21.109106,6.829393,0.721520,1.385963,-0.161764,2.323528
1,P-Test1,2022-04-27,-18.947193,-25.021257,-21.984225,6.074064,0.757244,1.320579,-0.138146,2.276292
2,P-Test1,2022-05-09,-16.746601,-22.163358,-19.454979,5.416757,0.755599,1.323454,-0.139213,2.278425
3,P-Test1,2022-05-21,-15.461473,-23.142317,-19.301895,7.680844,0.668104,1.496773,-0.198966,2.397932
4,P-Test1,2022-06-02,-17.303104,-24.044679,-20.673892,6.741575,0.719623,1.389616,-0.163046,2.326091
...,...,...,...,...,...,...,...,...,...,...
105,P-Test3,2022-12-23,-11.463176,-18.886185,-15.174680,7.423010,0.606961,1.647553,-0.244585,2.489171
106,P-Test3,2023-02-09,-11.437550,-17.938783,-14.688166,6.501233,0.637588,1.568411,-0.221309,2.442617
107,P-Test3,2023-02-21,-12.730329,-18.397605,-15.563967,5.667277,0.691956,1.445179,-0.182064,2.364128
108,P-Test3,2023-03-05,-11.513344,-16.200676,-13.857010,4.687332,0.710671,1.407122,-0.169132,2.338264


## Detect manure dates, using the best selected ML model (pre-trained)